In [0]:
import dlt
from pyspark.sql.functions import expr, sum as databricks_sum
# avoid using the default sum built-in, because the default sum takes an iterable as an argument where we'll be using the column name as string and then the built-in function won't be applied, so ERROR (you can try use the default/normal sum to test, but it won't work)

@dlt.table(
  table_properties={
    "quality" : "bronze",
    "pipelines.autoOptimize.managed": "true"
  },
  #partition_cols=[""],
  schema="""nome STRING, 
          cpf STRING, 
          orgao STRING, 
          cargo STRING, 
          funcao STRING, 
          situacao STRING, 
          mes STRING, 
          ano STRING, 
          codigo_do_orgao STRING, 
          matricula STRING, 
          remuneracao_basica STRING, 
          beneficios STRING, 
          valor_das_funcoes STRING, 
          comissao_conselheiro STRING, 
          hora_extra STRING, 
          verbas_eventuais STRING, 
          verbas_judiciais STRING, 
          descontos_a_maior STRING, 
          licenca_premio STRING, 
          irrf STRING, 
          seguridade_social STRING, 
          teto_redutor STRING, 
          outros_recebimentos STRING, 
          outros_descontos_obrigatorios STRING, 
          pagamento_a_maior STRING, bruto STRING, 
          liquido STRING,
          _rescued_data STRING""",
  temporary=False)
@dlt.expect_or_fail("table_not_null", "nome IS NOT NULL")
@dlt.expect_or_drop("have_remuneration", "liquido != ',00'")
def bronze_remuneracao():
  return (
    spark.readStream.format("cloudFiles") \
      .option("cloudFiles.format", "csv") \
      .option("inferColumnTypes", "false") \
      .option("multiLine", "true") \
      .option("delimiter", ";") \
      .option("header", "true") \
      .option("cloudFiles.schemaEvolutionMode", "rescue") \
      .option("encoding", "ISO-8859-1") \
      .load("/mnt/s3_dbfs/")
  )

In [0]:
@dlt.table(
  comment="Este conjunto de dados apresenta domínio de remuneração dos servidores do Governo do DF, detalhada por órgão e nome do servidor.",
  table_properties={
    "quality" : "silver",
    "pipelines.autoOptimize.managed": "true"
  },
  schema="""cpf STRING COMMENT 'NUMERO DO CADASTRO DE PESSOA FÍSICA MASCARADO', 
          orgao STRING COMMENT 'ÓRGÃO VINCULADO AO SERVIDOR', 
          cargo STRING COMMENT 'POSIÇÃO QUE O SERVIDOR OCUPA NO ÓRGÃO DE FORMA PERMANENTE', 
          funcao STRING COMMENT 'DESIGNAÇÃO TEMPORÁRIA PARA DESEMPENHO DE DETERMINADAS ATRIBUIÇÕES (CARGO COMISSIONADO)', 
          situacao STRING COMMENT 'CONDIÇÃO DO SERVIDOR EM RELAÇÃO AO EXERCÍCIO DE SUAS ATIVIDADES',
          mes INT COMMENT 'MÊS DE REFERÊNCIA',
          ano INT COMMENT 'ANO DE REFERÊNCIA',
          salario_bruto DOUBLE COMMENT 'VALOR BRUTO DA REMUNERAÇÃO',
          salario_liquido DOUBLE COMMENT 'VALOR LÍQUIDO APÓS DESCONTOS OBRIGATÓRIOS'""",
  temporary=False)
@dlt.expect_or_drop("person_exists", "cpf IS NOT NULL")
def silver_remuneracao():
  return (
    dlt.read_stream("bronze_remuneracao") \
      .withColumn("mes", expr("CAST(mes AS INT)")) \
      .withColumn("ano", expr("CAST(ano AS INT)")) \
      .withColumn("salario_bruto", expr("CAST(REPLACE(bruto, ',', '.') AS DOUBLE)")) \
      .withColumn("salario_liquido", expr("CAST(REPLACE(liquido, ',', '.') AS DOUBLE)")) \
      .select("cpf", "orgao", "cargo", "funcao", "situacao", "mes", "ano", "salario_bruto", "salario_liquido")
  )

In [0]:
@dlt.table(
  comment="Este conjunto de dados apresenta a agregação de valores de remuneração dos servidores do Governo do DF, detalhada por órgão e nome do servidor.",
  table_properties={
    "quality" : "gold",
    "pipelines.autoOptimize.managed": "true"
  },
  schema="""orgao STRING COMMENT 'ÓRGÃO VINCULADO AO SERVIDOR', 
          cargo STRING COMMENT 'POSIÇÃO QUE O SERVIDOR OCUPA NO ÓRGÃO DE FORMA PERMANENTE', 
          funcao STRING COMMENT 'DESIGNAÇÃO TEMPORÁRIA PARA DESEMPENHO DE DETERMINADAS ATRIBUIÇÕES (CARGO COMISSIONADO)', 
          situacao STRING COMMENT 'CONDIÇÃO DO SERVIDOR EM RELAÇÃO AO EXERCÍCIO DE SUAS ATIVIDADES',
          mes INT COMMENT 'MÊS DE REFERÊNCIA',
          ano INT COMMENT 'ANO DE REFERÊNCIA',
          total_salario_bruto DOUBLE COMMENT 'SOMATÓRIO DO TOTAL BRUTO DA REMUNERAÇÃO',
          total_salario_liquido DOUBLE COMMENT 'SOMATÓRIO DO TOTAL LÍQUIDO APÓS DESCONTOS OBRIGATÓRIOS'""",
  temporary=False)
def gold_remuneracaoo():
  return (
    dlt.read_stream("silver_remuneracao") \
      .groupBy("orgao", "cargo", "funcao", "situacao", "mes", "ano") \
      .agg(databricks_sum("salario_bruto").alias("total_salario_bruto"), databricks_sum("salario_liquido").alias("total_salario_liquido")) \
      .drop("salario_bruto", "salario_liquido")
  )  